In [1]:
!pip install matplotlib


In [2]:
!pip install nvcc4jupyter

In [3]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpm4l7cc7w".


In [4]:
!apt-get update
!apt-get install -y nvidia-cuda-toolkit
!apt-get install -y libopencv-dev


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,609 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,397 kB]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,459

In [168]:
%%writefile denoise.cu
#include <opencv2/opencv.hpp>
#include <cuda_runtime.h>
#include <iostream>
#include <cmath>

struct PixelDistance {
    float dist;
    int nx, ny;
};

// CUDA kernel
__global__ void knnDenoiseKernel(unsigned char* d_image, unsigned char* d_output, int width, int height, int K, float sigmaS, float sigmaR) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;

    if (x < width && y < height) {
        int index = (y * width + x) * 3; // RGB channels
        unsigned char pixel[3] = { d_image[index], d_image[index + 1], d_image[index + 2] };

        PixelDistance distances[9];
        int count = 0;

        // Calculate weights based on spatial and range Gaussian
        float gaussianWeights[9];
        float totalWeight = 0.0f;
        int halfWindow = 1; // 3x3 window

        for (int ky = -halfWindow; ky <= halfWindow; ky++) {
            for (int kx = -halfWindow; kx <= halfWindow; kx++) {
                int nx = x + kx;
                int ny = y + ky;

                if (nx >= 0 && nx < width && ny >= 0 && ny < height) {
                    int idx = (ny * width + nx) * 3;

                    float spatialDist = sqrtf(kx * kx + ky * ky);
                    float colorDist = sqrtf(
                        (pixel[0] - d_image[idx]) * (pixel[0] - d_image[idx]) +
                        (pixel[1] - d_image[idx + 1]) * (pixel[1] - d_image[idx + 1]) +
                        (pixel[2] - d_image[idx + 2]) * (pixel[2] - d_image[idx + 2])
                    );

                    // Gaussian weights for spatial and color
                    float wS = expf(-(spatialDist * spatialDist) / (2.0f * sigmaS * sigmaS));
                    float wR = expf(-(colorDist * colorDist) / (2.0f * sigmaR * sigmaR));
                    float weight = wS * wR;

                    gaussianWeights[count] = weight;
                    distances[count] = { colorDist, nx, ny };
                    totalWeight += weight;
                    count++;
                }
            }
        }

        // Sort distances based on color difference
        for (int i = 0; i < count - 1; i++) {
            for (int j = 0; j < count - i - 1; j++) {
                if (distances[j].dist > distances[j + 1].dist) {
                    PixelDistance temp = distances[j];
                    distances[j] = distances[j + 1];
                    distances[j + 1] = temp;
                }
            }
        }

        // Weighted average for K closest neighbors
        int sum[3] = {0, 0, 0};
        float weightSum = 0.0f;
        for (int i = 0; i < K && i < count; i++) {
            int nx = distances[i].nx;
            int ny = distances[i].ny;
            int idx = (ny * width + nx) * 3;

            float weight = gaussianWeights[i];
            sum[0] += d_image[idx] * weight;
            sum[1] += d_image[idx + 1] * weight;
            sum[2] += d_image[idx + 2] * weight;
            weightSum += weight;
        }

        d_output[index] = sum[0] / weightSum;
        d_output[index + 1] = sum[1] / weightSum;
        d_output[index + 2] = sum[2] / weightSum;
    }
}

int main() {
    cv::Mat image = cv::imread("noisy_image.jpg");
    if (image.empty()) {
        std::cerr << "Could not open or find the image!" << std::endl;
        return -1;
    }

    int width = image.cols;
    int height = image.rows;
    int K = 6;
    float sigmaS = 5.0f;
    float sigmaR = 15.0f;
    int iterations = 10; // Number of denoising iterations

    unsigned char *d_image, *d_output;
    cudaMalloc(&d_image, width * height * 3 * sizeof(unsigned char));
    cudaMalloc(&d_output, width * height * 3 * sizeof(unsigned char));

    cudaMemcpy(d_image, image.data, width * height * 3 * sizeof(unsigned char), cudaMemcpyHostToDevice);

    dim3 blockSize(16, 16);
    dim3 gridSize((width + blockSize.x - 1) / blockSize.x, (height + blockSize.y - 1) / blockSize.y);

    // Iterative denoising
    for (int i = 0; i < iterations; i++) {
        knnDenoiseKernel<<<gridSize, blockSize>>>(d_image, d_output, width, height, K, sigmaS, sigmaR);
        cudaDeviceSynchronize();

        // Copy result back for next iteration
        cudaMemcpy(d_image, d_output, width * height * 3 * sizeof(unsigned char), cudaMemcpyDeviceToDevice);
    }

    cv::Mat denoisedImage(height, width, CV_8UC3);
    cudaMemcpy(denoisedImage.data, d_output, width * height * 3 * sizeof(unsigned char), cudaMemcpyDeviceToHost);





    cv::imwrite("denoised_image.jpg", denoisedImage);
    cudaFree(d_image);
    cudaFree(d_output);

    std::cout << "Iterative denoising completed. Image saved as 'denoised_image.jpg'." << std::endl;
    return 0;
}

Overwriting denoise.cu


In [169]:
!nvcc -o denoise denoise.cu `pkg-config --cflags --libs opencv4` -w


In [170]:
!./denoise

Iterative denoising completed. Image saved as 'denoised_image.jpg'.
